# Práctica 1 - Clasificación de palabras

In [13]:
import pandas as pd
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import numpy as np
from sklearn.svm import SVC

## Preparación de los datos
En esta primera parte de la práctica vamos a preparar los datos para poder hacer la clasificación. Para ello haremos que cada muestra siga el siguiente formato: C1 | C2 | C3 | .. | CN | Y  (siendo C una característica y Y el target).

In [5]:
#Auxiliar function that reformats df to one that is more fitting to Classification Problems
def reformat(dataFrame):
    dataFrame['y']=dataFrame.columns[0]
    dataFrame.rename(columns={dataFrame.columns[0]: 'word'}, inplace=True)
    return dataFrame;
#RawData, it has been a little bit formatted before reading the csv, to facilitate the process.
raw=pd.read_csv("data/data.csv")
#Split df
catala, angles= raw.filter(['catala'], axis=1), raw.filter(['angles'], axis=1)
#Reformating
catala=reformat(catala)
angles=reformat(angles)

#Merging
wordsDF=pd.concat([catala,angles], axis=0)
#Shuffle the rows
wordsDF = wordsDF.sample(frac=1).reset_index(drop=True)



<bound method DataFrame.to_string of           word       y
0      suggest  angles
1         came  angles
2      planeta  catala
3     finestra  catala
4        saber  catala
...        ...     ...
1971       box  angles
1972     ready  angles
1973      amic  catala
1974   believe  angles
1975        am  angles

[1976 rows x 2 columns]>


### Caracterísiticas
Las características que hemos pensado que pueden ser útiles para la clasificación de las palabras son:
- Cantidad de caracteres (Númerica)
- Proporción de consonantes por vocal (consonantes / vocales) (Numérica)
- Contiene patrones o normas ortográficas de una lengua de las que vamos a clasificar?
    + Doble uso de vocal consecutivamente como es el caso del inglés (Categórica)
    + Acentos en caso de catalán (Categórica)
    + Contiene combinaciones de consonantes (consonant clusters) propias del inglés? (Categórica)

Referencias
- <Consonant_Clusters :https://www.aprendeinglessila.com/2013/09/consonantes-ingles-clusters/>
- <Frecuencia_De_Letras_Usadas_En_Catalan: https://es.sttmedia.com/frecuencias-de-letras-catalan>
- <Frecuencia_De_Letras_Usadas_En_Catalan: https://www3.nd.edu/~busiforc/handouts/cryptography/letterfrequencies.html>

Para añadir las columnas que representen estas características, hemos aplicado las siguientes funciones a las palabras.

In [8]:
#ratio de consonantes y vocales
def ratio (word):
    vocals=0
    for c in word:
        if isVocal(c):
            vocals+=1
    return vocals/len(word) 

#isVocal?
def isVocal(c):
    if(c=='a' or c=='e' or c=='i' or c=='o' or c=='u'):
        return True
    return 
#gotAccent?
def gotAccent(word):
    #List containing all possible accentuated chars from Catalan
    accentuatedChars=[ord('à'),ord('è'),ord('é'),ord('í'),ord('ò'),ord('ó'),ord('ú')]
    for c in word:
        if ord(c) in accentuatedChars:
            return 1
    return 0
def doubleVocal(word):
    ocurrences=["aa","ee","ii","oo","uu"]
    for oc in ocurrences:
        if word.find(oc)!=-1:
            return 1
    return 0

def enCC(word):
    ocurrences=["sch","spl","shr","squ","thr","spr","scr","sph","th","tw","sw","sk","sm"]
    for oc in ocurrences:
        if word.find(oc)!=-1:
            return 1
    return 0

#Features Adding
wordsDF['ratio']=wordsDF['word'].apply(ratio)
wordsDF['cantidadLetras']=wordsDF['word'].apply(len)
wordsDF['gotAccent']=wordsDF['word'].apply(gotAccent)
wordsDF['doubleVocal']=wordsDF['word'].apply(doubleVocal)
wordsDF['enCC']=wordsDF['word'].apply(enCC)
#Reorganize DF
wordsDF=wordsDF[['word','ratio','cantidadLetras','gotAccent','doubleVocal','enCC','y']]
wordsDF.to_csv('data/definitiveData.csv', index=False)
print(wordsDF.to_string)

<bound method DataFrame.to_string of           word     ratio  cantidadLetras  gotAccent  doubleVocal  enCC       y
0      suggest  0.285714               7          0            0     0  angles
1         came  0.500000               4          0            0     0  angles
2      planeta  0.428571               7          0            0     0  catala
3     finestra  0.375000               8          0            0     0  catala
4        saber  0.400000               5          0            0     0  catala
...        ...       ...             ...        ...          ...   ...     ...
1971       box  0.333333               3          0            0     0  angles
1972     ready  0.400000               5          0            0     0  angles
1973      amic  0.500000               4          0            0     0  catala
1974   believe  0.571429               7          0            0     0  angles
1975        am  0.500000               2          0            0     0  angles

[1976 rows x 7

## Modelos
En este problema creemos que el mejor modelo para clasificar las palabras va a ser una SVC lineal, ya que solamente debemos de clasificar entre dos clases. El kernel a usar podría ser uno lineal, aunque el RBF podría ser buena opción ya que tenemos un número de características bastante bajo en comparación al número de muestras. 

### Separación del conjunto de datos en entrenamiento y test
Vamos a escoger un 2/3 para el conjunto de entrenamiento y el resto para el test. 


In [15]:
#First we must separate dataframe into X and y format
X=wordsDF.iloc[:,:6]
y=wordsDF.iloc[:,6]

#Then we separate the data frame in training and test 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=33)

         word     ratio  cantidadLetras  gotAccent  doubleVocal  enCC
489      cura  0.500000               4          0            0     0
931    beauty  0.500000               6          0            0     0
119       get  0.333333               3          0            0     0
611       cel  0.333333               3          0            0     0
721     check  0.200000               5          0            0     0
...       ...       ...             ...        ...          ...   ...
658    simple  0.333333               6          0            0     0
578      tant  0.250000               4          0            0     0
1752   parlar  0.333333               6          0            0     0
391   instant  0.285714               7          0            0     0
1044   liquid  0.500000               6          0            0     0

[1323 rows x 6 columns]
            word     ratio  cantidadLetras  gotAccent  doubleVocal  enCC
822        track  0.200000               5          0         

Los híperparámetros que deberemos de configurar en el modelo del SVM Lineal son:
- _penalty_
- _loss_
- _C_
- _max_iter_

El resto de híperparámetros no son tan necesarios ya que no se ajustan al problema. Por ejemplo _dual_  no nos va a dar ninguna diferencia, ya que es un booleano que indica que algoritmo usar. Se usa **dual = False** cuando el número de muestras es mayor al de características, que es nuestro caso. <br>Tampoco usaremos *multi_class* ya que en este experimento solo usaremos las clases "catalan" y "angles".